### Очень усердные ученики

**Условие**

Образовательные курсы состоят из различных уроков, каждый из которых состоит из нескольких маленьких заданий. Каждое такое маленькое задание называется "горошиной".

Назовём очень усердным учеником того пользователя, который хотя бы раз за текущий месяц правильно решил 20 горошин.

**Задача**

Дана таблица **default.peas**:

| Название атрибута | Тип атрибута | Смысловое значение |
|:--------:|:--------:|:--------:|
|  st_id     |  int         |  ID ученика   |
|  timest    |  timestamp   |  	Время решения карточки   |
|  correct   |  bool        |  Правильно ли решена горошина?   |
|  subject   |  text        |  	Дисциплина, в которой находится горошина   |

Необходимо написать оптимальный запрос, который даст информацию о количестве очень усердных студентов.

NB! Под усердным студентом мы понимаем студента, который правильно решил 20 задач за текущий месяц.

In [1]:
import pandahouse as ph

gодключимся к ClIckHouse через pandahouse.

In [2]:
connection_default = {'host': 'https://clickhouse.lab.karpov.courses',
                      'database':'default',
                      'user':'student', 
                      'password':'dpo_python_2020'
                     }

Посмотрим на данные - выведем первые 5 строк таблицы default.peas. Тип данных столбца timest преобразуем в DateTime.

In [3]:
q = '''
    SELECT st_id, CAST(timest AS DateTime) AS timest,correct, subject
    FROM default.peas
'''
q_test = ph.read_clickhouse(query=q, connection=connection_default)
q_test.head()

,st_id,timest,correct,subject
0,100379,2021-10-30 13:32:29,1,Theory of probability
1,100379,2021-10-30 14:11:19,0,Vizualization
2,100379,2021-10-30 15:54:22,1,Theory of probability
3,100379,2021-10-30 16:44:50,1,Vizualization
4,100379,2021-10-30 17:15:05,1,Theory of probability


Проверим, за какой отрезок времени наши данные.

In [4]:
q_time = '''
    SELECT MIN(CAST(timest AS DateTime)) AS min_timest, MAX(CAST(timest AS DateTime)) AS max_timest
    FROM default.peas
'''
q_test_time = ph.read_clickhouse(query=q_time, connection=connection_default)
q_test_time

,min_timest,max_timest
0,2021-10-30 13:00:00,2021-10-31 17:29:52


Данные всего за 2 дня - 30 и 31 октября 2021 года.

Перейдем к выполнению первого задания.

In [5]:
#используем подзапрос - в нем выбираем id усердных студентов
q_1 = '''
    SELECT COUNT(DISTINCT st_id) AS strong_students
    FROM (
        SELECT st_id
        FROM default.peas
        WHERE correct = True
        GROUP BY st_id
        HAVING COUNT(correct) >= 20
        )
'''

q_strong_students = ph.read_clickhouse(query=q_1, connection=connection_default)
q_strong_students

,strong_students
0,136


**Ответ**: 136 учеников являются очень усердными.

### Оптимизация воронки

**Условие**

Образовательная платформа предлагает пройти студентам курсы по модели trial: студент может решить бесплатно лишь 30 горошин в день. Для неограниченного количества заданий в определенной дисциплине студенту необходимо приобрести полный доступ. Команда провела эксперимент, где был протестирован новый экран оплаты.

**Задача**

Дана таблицы: **default.peas** (см. выше), **default.studs**:

| Название атрибута | Тип атрибута | Смысловое значение |
|:--------:|:--------:|:--------:|
|  st_id      |  int    |  ID ученика   |
|  test_grp   |  text   |  Метка ученика в данном эксперименте   |


и **default.final_project_check**:

| Название атрибута | Тип атрибута | Смысловое значение |
|:--------:|:--------:|:--------:|
|  st_id        |  int         |  ID ученика   |
|  sale_time    |  timestamp   |  		Время покупки   |
|  money        |  int         |  Цена, по которой приобрели данный курс   |
|  subject      |  text        |  	Дисциплина, в которой находится горошина   	

Необходимо в одном запросе выгрузить следующую информацию о группах пользователей:

- ARPU 
- ARPAU 
- CR в покупку 
- СR активного пользователя в покупку 
- CR пользователя из активности по математике (subject = ’math’) в покупку курса по математике

ARPU считается относительно всех пользователей, попавших в группы.

Активным считается пользователь, за все время решивший больше 10 задач правильно в любых дисциплинах.

Активным по математике считается пользователь, за все время решивший 2 или больше задач правильно по математике.

In [6]:
q_test.head()
# default.peas

,st_id,timest,correct,subject
0,100379,2021-10-30 13:32:29,1,Theory of probability
1,100379,2021-10-30 14:11:19,0,Vizualization
2,100379,2021-10-30 15:54:22,1,Theory of probability
3,100379,2021-10-30 16:44:50,1,Vizualization
4,100379,2021-10-30 17:15:05,1,Theory of probability


Тоже посмотрим на данные - выведем первые 5 строк таблиц default.studs и default.final_project_check. Тип данных столбца sale_time в таблице default.final_project_check тоже преобразуем в DateTime.

In [7]:
q_2 = '''
    SELECT *
    FROM default.studs
'''
q_test_2 = ph.read_clickhouse(query=q_2, connection=connection_default)
q_test_2.head()

,st_id,test_grp
0,100379,pilot
1,101432,control
2,104818,pilot
3,104885,pilot
4,104966,pilot


In [8]:
q_3 = '''
    SELECT st_id, CAST(sale_time AS DateTime) AS sale_time, money, subject
    FROM default.final_project_check
'''
q_test_3 = ph.read_clickhouse(query=q_3, connection=connection_default)
q_test_3.head()

,st_id,sale_time,money,subject
0,101432,2021-10-31 04:44:32,85000,Math
1,101432,2021-10-31 12:43:50,65000,Vizualization
2,104885,2021-10-30 17:05:55,65000,Vizualization
3,104885,2021-10-30 22:49:33,75000,Statistics
4,106464,2021-10-31 13:17:13,85000,Math


Здесь тоже на всякий случай проверим, за какой период времени данные.

In [9]:
q_3_time = '''
    SELECT MIN(CAST(sale_time AS DateTime)) AS min_sale_time, MAX(CAST(sale_time AS DateTime)) AS max_sale_time
    FROM default.final_project_check
'''
q_test_3_time = ph.read_clickhouse(query=q_3_time, connection=connection_default)
q_test_3_time

,min_sale_time,max_sale_time
0,2021-10-30 15:46:33,2021-10-31 15:15:58


Те же дни - 30 и 31 октября 2021 года.

### Метрики

1. **ARPU** = доход от всех пользователей / число всех пользователей
    
2. **ARPAU** = доход от активных пользователей / число активных пользователей
    
3. **CR в покупку** = (число пользователей, которые сделали покупку / число всех пользователей) * 100
    
4. **СR активного пользователя в покупку** = (число активных пользователей, которые сделали покупку / число активных пользователей) * 100
    
5. **CR пользователя из активности по математике в покупку курса по математике** = (число активных пользователей по мат., которые сделали покупку курса по математике / число активных пользователей по математике) * 100

Найдем все необходимые переменные для расчета метрик. Далее - создадим новую таблицу с посчитанными метриками, все полученные запросы для расчета метрик объеденим с помощью левого джоина.

In [10]:
q_4 = '''
WITH
    -- доход от всех пользователей
    revenue_total AS
    (
    SELECT test_grp, SUM(money) AS money
    FROM default.studs
    LEFT JOIN default.final_project_check
    ON default.studs.st_id = default.final_project_check.st_id
    GROUP BY test_grp
    ),
    
    -- число всех пользователей
    all AS
    (
    SELECT test_grp, COUNT(DISTINCT st_id) AS students
    FROM default.studs
    GROUP BY test_grp
    ),
    
    -- доход от активных пользователей
    revenue_active AS
    (
    SELECT test_grp, SUM(money) AS money
    FROM (
            SELECT test_grp, st_id
            FROM (
                    SELECT st_id, SUM(correct)
                    FROM default.peas
                    GROUP BY st_id
                    HAVING SUM(correct) > 10) AS q
            LEFT JOIN default.studs
            ON default.studs.st_id = q.st_id) AS act_stud
    INNER JOIN default.final_project_check
    ON act_stud.st_id = default.final_project_check.st_id
    GROUP BY test_grp
    ),
    
    -- число активных пользователей
    active AS
    (
    SELECT test_grp, COUNT(DISTINCT st_id) AS students
    FROM (
                SELECT st_id, SUM(correct)
                FROM default.peas
                GROUP BY st_id
                HAVING SUM(correct) > 10) AS q
    LEFT JOIN default.studs
    ON default.studs.st_id = q.st_id
    GROUP BY test_grp
    ),
    
    -- число пользователей, которые сделали покупку
    all_paying AS 
    (
    SELECT test_grp, COUNT(DISTINCT st_id) AS students
    FROM default.studs
    INNER JOIN default.final_project_check
    ON default.studs.st_id = default.final_project_check.st_id
    GROUP BY test_grp
    ),
    
    -- число активных пользователей, которые сделали покупку
    active_paying AS
    (
    SELECT test_grp, COUNT(DISTINCT st_id) AS students
    FROM (
            SELECT test_grp, st_id
            FROM (
                    SELECT st_id, SUM(correct)
                    FROM default.peas
                    GROUP BY st_id
                    HAVING SUM(correct) > 10) AS q
            LEFT JOIN default.studs
            ON default.studs.st_id = q.st_id) AS act_stud
    INNER JOIN default.final_project_check
    ON act_stud.st_id = default.final_project_check.st_id
    GROUP BY test_grp
    ),
    
    -- число активных пользователей по математике
    math_active AS
    (
    SELECT test_grp, COUNT(DISTINCT st_id) AS students
    FROM default.studs
    INNER JOIN ( 
                SELECT st_id, SUM(correct)
                FROM default.peas
                WHERE subject = 'Math'
                GROUP BY st_id
                HAVING SUM(correct) >= 2) AS q_math
    ON default.studs.st_id = q_math.st_id
    GROUP BY test_grp
    ),
    
    -- число активных пользователей по математике, которые сделали покупку курса по математике
    math_active_paying AS 
    (
    SELECT test_grp, COUNT(DISTINCT st_id) AS students
    FROM (
            SELECT test_grp, st_id
            FROM default.studs
            INNER JOIN ( 
                    SELECT st_id, SUM(correct)
                    FROM default.peas
                    WHERE subject = 'Math'
                    GROUP BY st_id
                    HAVING SUM(correct) >= 2) AS q_math
            ON default.studs.st_id = q_math.st_id
            ) AS act_math_stud
    INNER JOIN default.final_project_check
    ON act_math_stud.st_id = default.final_project_check.st_id
    WHERE subject = 'Math'
    GROUP BY test_grp
    )
        
SELECT
    revenue_total.test_grp,
    ROUND((revenue_total.money / all.students), 2) AS ARPU,
    ROUND((revenue_active.money / active.students), 2) AS ARPAU,
    ROUND((all_paying.students / all.students) * 100, 2) AS CR,
    ROUND((active_paying.students / active.students) * 100, 2) AS CR_ACTIVE,
    ROUND((math_active_paying.students / math_active.students) * 100, 2) AS CR_MATH
            
FROM all
LEFT JOIN revenue_total
ON all.test_grp = revenue_total.test_grp
        
LEFT JOIN revenue_active
ON revenue_total.test_grp = revenue_active.test_grp
        
LEFT JOIN active
ON revenue_active.test_grp = active.test_grp
        
LEFT JOIN all_paying
ON active.test_grp = all_paying.test_grp
        
LEFT JOIN active_paying
ON all_paying.test_grp = active_paying.test_grp
        
LEFT JOIN math_active
ON active_paying.test_grp = math_active.test_grp
        
LEFT JOIN math_active_paying
ON math_active.test_grp = math_active_paying.test_grp
'''
q_test_4 = ph.read_clickhouse(query=q_4, connection=connection_default)
q_test_4.head()

,revenue_total.test_grp,ARPU,ARPAU,CR,CR_ACTIVE,CR_MATH
0,control,4540.98,10393.70,4.92,11.02,6.12
1,pilot,11508.47,29739.58,10.85,26.04,9.52
